# 卷积神经网络(CNN)  mnist数据集
`凌鑫杰`

_导入模块_

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
from torchvision import datasets, transforms
import numpy as np
# % matplotlib inline

In [3]:
##定义超参数
input_size = 28  #图形总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 3  #训练总循环周期
batch_size = 64  #一个撮(批次)的大小，64张图片
##训练集
train_dataset = datasets.MNIST(root="./data", train=True, transform=transforms.ToTensor(), download=True)

##测试集
test_dataset = datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor())

##构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# 定义卷积神经网络模型

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_1 = nn.Sequential(  #输入大小
            nn.Conv2d(
                in_channels=1,  #灰度图
                out_channels=16,  #卷积核个数(特征图个数)
                kernel_size=5,  #卷积核大小(选择区域大小)
                stride=1,  #滑动窗口步长
                padding=2,  #边缘填充
            ),  #输出特征图为(16,28,28)
            nn.ReLU(),  #RELU层(激活层)
            nn.MaxPool2d(kernel_size=2),  #进行池化操作，输出结果：(16,14,14)
        )
        self.conv_2 = nn.Sequential(  #第一层卷积的输入(16,14,14)
            nn.Conv2d(16, 32, 5, 1, 2),  #输出(32,14,14)
            nn.ReLU(),  #RULU层(激活层)
            nn.MaxPool2d(2),  #输出(32,7,7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)  #全连接层得到的结果

    def forward(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [5]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [6]:
#实例化
net = CNN()
#损失函数
criterion = nn.CrossEntropyLoss()
#优化器
optimizer = optim.Adam(net.parameters(), lr=0.001)  #定义优化器，普通的随机梯度下降算法

#开始训练循环
for epoch in range(num_epochs):
    #当前epoch结果保存下来
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_loader):  #针对容器中每一个批进行循环
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)

        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []

            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)

            #准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print("当前epoch：{}[{}/{}({:.0f}%)]\t损失：{:.6f}\t训练集准确率：{:.2f}%\t测试集正确率：{:.2f}%".format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                loss.data,
                       100. * train_r[0].numpy() / train_r[1],
                       100. * val_r[0].numpy() / val_r[1]
            ))



当前epoch：0[0/60000(0%)]	损失：2.299800	训练集准确率：12.50%	测试集正确率：10.56%
当前epoch：0[6400/60000(11%)]	损失：0.270555	训练集准确率：76.05%	测试集正确率：90.80%
当前epoch：0[12800/60000(21%)]	损失：0.144860	训练集准确率：84.65%	测试集正确率：94.68%
当前epoch：0[19200/60000(32%)]	损失：0.110791	训练集准确率：87.99%	测试集正确率：95.93%
当前epoch：0[25600/60000(43%)]	损失：0.097812	训练集准确率：89.94%	测试集正确率：96.73%
当前epoch：0[32000/60000(53%)]	损失：0.101021	训练集准确率：91.21%	测试集正确率：97.33%
当前epoch：0[38400/60000(64%)]	损失：0.091958	训练集准确率：92.15%	测试集正确率：97.50%
当前epoch：0[44800/60000(75%)]	损失：0.025826	训练集准确率：92.92%	测试集正确率：97.73%
当前epoch：0[51200/60000(85%)]	损失：0.016574	训练集准确率：93.53%	测试集正确率：97.90%
当前epoch：0[57600/60000(96%)]	损失：0.074326	训练集准确率：93.98%	测试集正确率：98.09%
当前epoch：1[0/60000(0%)]	损失：0.171017	训练集准确率：95.31%	测试集正确率：98.19%
当前epoch：1[6400/60000(11%)]	损失：0.041638	训练集准确率：98.17%	测试集正确率：98.18%
当前epoch：1[12800/60000(21%)]	损失：0.045806	训练集准确率：98.09%	测试集正确率：98.10%
当前epoch：1[19200/60000(32%)]	损失：0.090600	训练集准确率：98.08%	测试集正确率：98.56%
当前epoch：1[25600/60000(43%)]	损失：0.122465	训练集准确率：98.16%	测试集正确率

In [7]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()